In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
customer = "Tchibo"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.performance_review" #features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path)#, limit=200000)

In [5]:
df.sort_values(by="date",ascending=False)

In [6]:
df["date"].min(), df["date"].max()

In [7]:
df["date"].value_counts().sort_index(ascending=False)

In [8]:
unique_treatments = df["treatment"].dropna().unique().tolist()
len(unique_treatments)

In [9]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":unique_treatments},
    logging
)
treatments = pd.json_normalize(treatments)

In [10]:
treatments["source"].value_counts()

In [11]:
df[df["date"]==df["date"].max()]